In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
os.chdir('E:\Machine Learning Course\Course3')

In [ ]:
df = pd.read_csv('aug_train.csv')

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
cata = df.columns[df.dtypes==np.object]

In [ ]:
cata

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
for x in cata:
    df[x] = le.fit_transform(df[x])

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.drop('id',axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
target = 'Response'
feature_cols = [x for x in df if x!=target]

In [ ]:
sns.pairplot(df)

In [ ]:
plt.figure(figsize=(40,40))
sns.heatmap(df.corr(method ='pearson'),cmap='PuBu',annot=True,linewidths=0.5)
plt.show()

In [ ]:
skew_limit = 0.75
skew_vals = df[feature_cols].skew()
skew_vals

In [ ]:
skew_cols = (skew_vals.sort_values(ascending=False).to_frame().rename(columns={0:'Skew'}).query('abs(Skew) > {}'.format(skew_limit)))
skew_cols

In [ ]:
sns.histplot(df['Annual_Premium'],bins=25)

In [ ]:
sns.histplot(df['Policy_Sales_Channel'],bins=25)

In [ ]:
sns.histplot(df['Driving_License'],bins=25)

In [ ]:
for x in skew_cols.index.values:
    if x == "price":
        continue
    df[x] = df[x].apply(np.log1p)

In [ ]:
skew_vals = df[feature_cols].skew()
skew_vals

In [ ]:
sns.histplot(df['Annual_Premium'],label='Annual_Premium', palette='bone');


In [ ]:
sns.histplot(df['Policy_Sales_Channel'], label='Policy_Sales_Channel');


In [ ]:
df[target].value_counts(normalize=True)

In [ ]:
test = pd.read_csv('aug_test.csv')

In [ ]:
test.head(3)

In [ ]:
test.shape

In [ ]:
cols = test.columns[test.dtypes==np.object]
cols

In [ ]:
for x in cols:
    test[x] = le.fit_transform(test[x])

In [ ]:
test.info()

In [ ]:
test.drop('id',axis=1,inplace=True)

In [ ]:
test.info()

In [ ]:
test_mask = test.columns!=np.object

In [ ]:
test_cols = test.columns[test_mask]
test_cols

In [ ]:
test_skew_vals = test[test_cols].skew()

In [ ]:
test_skew_vals

In [ ]:
test_skew_cols = (test_skew_vals.sort_values(ascending=False).to_frame().rename(columns={0:"Skew"}).query('abs(Skew)>{}'.format(skew_limit)))
test_skew_cols

In [ ]:
for x in test_skew_cols.index.values:
    test[x] = test[x].apply(np.log1p)

In [ ]:
test.describe().T

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
mm = MinMaxScaler()
ss = StandardScaler()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split

In [ ]:
X = df[feature_cols]
y = df[target]

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [ ]:
from sklearn.metrics import *

In [ ]:
lr = LogisticRegression()
svc = SVC()
rf = RandomForestClassifier()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,stratify=y)

In [ ]:
svc = svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)

In [ ]:
accuracy = []
for folds,(train_idx,test_idx) in enumerate(skf.split(X,y)):
    X_train,X_test,y_train,y_test = (X.iloc[train_idx,:],X.iloc[test_idx,:],y[train_idx],y[test_idx])
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    svc = svc.fit(X_train,y_train)
    y_pred = svc.predict(X_test)
    ac = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
accuracy = []
for folds,(train_idx,test_idx) in enumerate(skf.split(X,y)):
    X_train,X_test,y_train,y_test = (X.iloc[train_idx,:],X.iloc[test_idx,:],y[train_idx],y[test_idx])
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    rf = rf.fit(X_train,y_train)
    y_pred = rf.predict(X_test)
    ac = accuracy_score(y_test,y_pred)
    accuracy.append(ac)
accuracy

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [ ]:
param = {'n_estimators':[int(x) for x in np.linspace(start = 5 , stop = 15, num = 10)],
             'max_features':['auto','log2'],
             'max_depth':[int(x) for x in np.linspace(5, 10, num = 2)]}

In [ ]:
smt = SMOTE(sampling_strategy='minority', k_neighbors=5)
cmb = SMOTETomek()

In [ ]:
accuracy = []
for folds,(train_idx,test_idx) in enumerate(skf.split(X,y)):
    X_train,X_test,y_train,y_test = (X.iloc[train_idx,:],X.iloc[test_idx,:],y[train_idx],y[test_idx])
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    train_u, y_train_u = smt.fit_resample(X_train, y_train)
    rf = GridSearchCV(estimator=rf,param_grid=param,scoring='accuracy',n_jobs=-1)
    rf = rf.fit(X_train,y_train)
    y_pred = rf.predict(X_test)
    ac = accuracy_score(y_test,y_pred)
    accuracy.append(pd.Series({'Estimator':rf.best_estimator_,'accuracy':ac}))
accuracy

In [ ]:
accuracy = []

param = {
    'n_estimators':[15,20,40,50,100,150,200,300,400],
    'max_features':['auto','log2'],
    'max_depth':[5,7,11,25]
}
for folds,(train_idx,test_idx) in enumerate(skf.split(X,y)):
    X_train,X_test,y_train,y_test = (X.iloc[train_idx,:],X.iloc[test_idx,:],y[train_idx],y[test_idx])
    grid = GridSearchCV(estimator=rf,param_grid=param,cv=skf)
    grid.fit(X_train,y_train)

In [ ]:
param = {
    'n_estimators':[15,20,40,50,100,150,200,300,400],
    'max_features':['auto','log2'],
    'max_depth':[5,7,11,25]
}
for folds,(train_idx,test_idx) in enumerate(skf.split(X,y)):
    X_train,X_test,y_train,y_test = (X.iloc[train_idx,:],X.iloc[test_idx,:],y[train_idx],y[test_idx])
    grid = GridSearchCV(estimator=RandomForestClassifier(),param_grid=param)
    grid.fit(X_train,y_train)